In [ ]:
import zipfile
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
  zip_ref.extractall("")

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd


def extract_features(img):
    return  list(cv2.mean(im))[:3]


data=[]
basepath='./dataset/'
for folder in os.scandir(basepath):
    if folder.is_dir():
        #print(folder.name)
        for file in os.scandir(folder.path):
            im=cv2.imread(file.path)
            im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
            features=extract_features(im)
            features.append(folder.name)
            data.append(features)
            #print(features)

In [ ]:
print(data)

In [ ]:
df = pd.DataFrame(data, columns=['R','G','B','class'])
df.to_csv('curdata.csv',index=False)

In [ ]:
df = pd.read_csv('curdata.csv')
print(df)

In [ ]:
lX = df[df['class']=='lemon']
mX = df[df['class']=='melon']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 10), dpi= 80, facecolor='w', edgecolor='k')
ax = plt.axes(projection='3d')

ax.set_xlabel('R')
ax.set_ylabel('G')
ax.set_zlabel('B');

ax.scatter3D(lX['R'], lX['G'], lX['B'],color='y')
ax.scatter3D(mX['R'], mX['G'], mX['B'],color='g')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X)
print(y)

le = preprocessing.LabelEncoder()
y=le.fit(y).transform(y)

print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(random_state=42, hidden_layer_sizes=(100,50),max_iter=1000, solver='adam')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


In [ ]:
import urllib.request

def get_image(url):

  # Open the URL and read the image data
  with urllib.request.urlopen(url) as response:
      image_data = response.read()

  # Decode the image data and convert it to a NumPy array
  image_array = np.array(bytearray(image_data), dtype=np.uint8)

  # Decode the image array into an OpenCV image
  image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
  return image

In [ ]:

import matplotlib.pyplot as plt

url=input("Enter image url : ")
im=get_image(url)
im=cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
features=extract_features(im)
yp=classifier.predict([features])
yp=le.inverse_transform(yp)

print('It is a '+yp[0])
plt.axis("off")
plt.imshow(im)
plt.show()